In [1]:
import numpy as np
import scipy
from scipy.sparse import csc_matrix, csr_matrix, lil_matrix
import sparse

from nmrtools.nmrmath import is_allowed, normalize_spectrum, transition_matrix
from nmrtools.nmrplot import nmrplot
from speedtest.compare_hamiltonians import hamiltonian, hamiltonian_sparse
from speedtest.speedutils import timefn

from simulation_data import spin8, spin3, spin11, rioux

In [2]:
H3_MATRIX = hamiltonian(*spin3())
H3_NDARRAY = hamiltonian_sparse(*spin3())
H8_MATRIX = hamiltonian(*spin8())
H8_NDARRAY = hamiltonian_sparse(*spin8())
H_RIOUX = hamiltonian(*rioux())

 2019-04-27 13:33:29,865 - INFO @timefn:hamiltonian_sparse took 0.4295768737792969 seconds
 2019-04-27 13:33:29,895 - INFO @timefn:hamiltonian_sparse took 0.011005163192749023 seconds


@timefn:hamiltonian_sparse took 0.4295768737792969 seconds
@timefn:hamiltonian_sparse took 0.011005163192749023 seconds


In [3]:
@timefn
def numpy_eigh(h):
    return np.linalg.eigh(h)


@timefn
def scipy_eigh(h):
    return scipy.linalg.eigh(h)

In [4]:
def matrix_eigh(h, n):
    for i in range(n):
        E1, V1 = numpy_eigh(h)
        E2, V2 = scipy_eigh(h)
    return (E1, V1), (E2, V2)

In [36]:
matrix_numpy, matrix_scipy = matrix_eigh(H_RIOUX, 1)
array_numpy, array_scipy = matrix_eigh(H_RIOUX, 1)
_, V_matrix_numpy = matrix_numpy
_, V_matrix_scipy = matrix_scipy
_, V_array_numpy = array_numpy
_, V_array_scipy = array_scipy
V = [V_matrix_numpy, V_matrix_scipy, V_array_numpy, V_array_scipy]
V_real = [i.real for i in V]
for i, j in zip(V, V_real):
    assert np.allclose(i, j)

 2019-04-27 17:24:13,539 - INFO @timefn:numpy_eigh took 0.001003265380859375 seconds
 2019-04-27 17:24:13,540 - INFO @timefn:scipy_eigh took 0.0 seconds
 2019-04-27 17:24:13,540 - INFO @timefn:numpy_eigh took 0.0 seconds
 2019-04-27 17:24:13,541 - INFO @timefn:scipy_eigh took 0.0 seconds


@timefn:numpy_eigh took 0.001003265380859375 seconds
@timefn:scipy_eigh took 0.0 seconds
@timefn:numpy_eigh took 0.0 seconds
@timefn:scipy_eigh took 0.0 seconds


In [21]:
def test_ev_matching():
    numpy_result, scipy_result = matrix_eigh(H_RIOUX, 1)
    E1, V1 = numpy_result
    V1_asarray = np.array(V1)
    print(type(V1))
    print(type(V1_asarray))
    E2, V2 = scipy_result
    print(type(V2))
    assert not np.allclose(V1[:,1], V2[:, 1])
    assert np.allclose(V1_asarray[:,1], V2[:,1])
    # The different eigenvector solutions have terms differing in sign, which
    # apparently is OK. So, testing functional equality by using squares::
    V1_ = np.square(V1)
    V2_ = np.square(V2)
    print(V1_)
    print(V2_)
    assert np.allclose(V1_, V2_)

In [23]:
# test_ev_matching()

In [30]:
def transition_matrix(n):
    """
    Creates a matrix of allowed transitions.

    The integers 0-`n`, in their binary form, code for a spin state
    (alpha/beta). The (i,j) cells in the matrix indicate whether a transition
    from spin state i to spin state j is allowed or forbidden.
    See the ``is_allowed`` function for more information.

    Parameters
    ---------
    n : dimension of the n,n matrix (i.e. number of possible spin states).

    Returns
    -------
    lil_matrix
        a transition matrix that can be used to compute the intensity of
    allowed transitions.
    """
    # function was optimized by only calculating upper triangle and then adding
    # the lower.
    T = lil_matrix((n, n))  # sparse matrix created
    print(type(T), ' before operations')
    for i in range(n - 1):
        for j in range(i + 1, n):
            if is_allowed(i, j):
                T[i, j] = 1
    print(type(T), ' before adding transpose')
    T = T + T.T
    return T

In [31]:
print(type(transition_matrix(8)))

<class 'scipy.sparse.lil.lil_matrix'>  before operations
<class 'scipy.sparse.lil.lil_matrix'>  before adding transpose
<class 'scipy.sparse.csr.csr_matrix'>


In [19]:
print(V1)
print(V2)
print(type(V1))
print(type(V2))

[[ 0.          0.          0.          0.          0.          0.
   0.          1.        ]
 [ 0.          0.          0.          0.          0.05607604  0.99815514
  -0.02327631  0.        ]
 [ 0.          0.          0.          0.          0.02004006 -0.02443353
  -0.99950057  0.        ]
 [ 0.          0.02119554 -0.05911396 -0.9980262   0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.         -0.99822536  0.05558157
  -0.02137322  0.        ]
 [ 0.         -0.99959476  0.01771957 -0.0222784   0.          0.
   0.          0.        ]
 [ 0.          0.01900156  0.99809396 -0.05871443  0.          0.
   0.          0.        ]
 [ 1.          0.          0.          0.          0.          0.
   0.          0.        ]]
[[ 0.          0.          0.          0.          0.          0.
   0.          1.        ]
 [ 0.          0.          0.          0.          0.05607604  0.99815514
   0.02327631  0.        ]
 [ 0.          0.          0.        

In [6]:
test_ev_matching()

 2019-04-27 13:34:34,821 - INFO @timefn:numpy_eigh took 0.002997159957885742 seconds
 2019-04-27 13:34:34,822 - INFO @timefn:scipy_eigh took 0.0 seconds


@timefn:numpy_eigh took 0.002997159957885742 seconds
@timefn:scipy_eigh took 0.0 seconds
(8, 8)
(8, 8)
-491.625
[[0. 0. 0. 0. 0. 0. 0. 1.]]
--------------------
-230.96347142517027
[[ 0.          0.          0.          0.          0.05607604  0.99815514
  -0.02327631  0.        ]]
--------------------
-200.30588633096843
[[ 0.          0.          0.          0.          0.02004006 -0.02443353
  -0.99950057  0.        ]]
--------------------
-72.10564224386131
[[ 0.          0.02119554 -0.05911396 -0.9980262   0.          0.
   0.          0.        ]]
--------------------
61.88341711576846
[[ 0.          0.          0.          0.         -0.99822536  0.05558157
  -0.02137322  0.        ]]
--------------------
195.52427326502007
[[ 0.         -0.99959476  0.01771957 -0.0222784   0.          0.
   0.          0.        ]]
--------------------
234.21730961921148
[[ 0.          0.01900156  0.99809396 -0.05871443  0.          0.
   0.          0.        ]]
--------------------
503.375
[[

AssertionError: 